## 问题思路：
### 数据预处理->数据分析->选取特征->模型构建与训练

### 一、问题定义：

1、背景：自行车共享系统是一种租赁自行车的方式，其中获得会员资格、租赁和归还自行车的过程都是通过遍布城市的kiosk位置网络自动完成的。使用这些系统，人们可以从一个地方租一辆自行车，并根据需要把它送回不同的地方。目前，全世界有超过500个共享单车项目。


这些系统产生的数据使它们对研究人员具有吸引力，因为旅行的持续时间、出发地点、到达地点和经过的时间都被明确地记录下来。因此，自行车共享系统就像一个传感器网络，可以用来研究城市中的流动性。

2、目的：在这场比赛中，参与者被要求结合历史使用模式和天气数据，以预测华盛顿特区首都自行车共享项目的自行车租赁需求。

3、问题转化：本问题的训练集给出了一系列特征值和用户租赁共享单车数量的数据，经过模型的构建和训练，最终通过测试集中的天气等特征值预测会员租赁数量，临时租赁数量和总租赁数量，是一个经典的机器学习中的监督学习中的预测问题。

### 二、数据预处理与数据分析

#### 1、数据说明：
（1）数据概况：比赛提供了跨越两年的每小时租赁数据，包含天气信息和日期信息，训练集由每月前19天的数据组成，测试集是每月第二十天到当月底的数据。


（2）变量说明：

|  变量名   | 变量说明  |
|  ----  | ----  |
| datetime  | 日期+时间 |
| season  | 1,2,3,4=春,夏,秋,冬 |
| holiday  | 是否是节假日 |
| workingday  | 1,0=工作日,周末 |
| weather  | 详细说明 |
| temp  | 气温摄氏度 |
| atemp  | 体感温度 |
| humidity | 湿度 |
| windspeed  | 风速 |
| casual  | 非注册用户个数 |
| registered  | 注册用户个数 |
| count  | 给定日期时间(每小时)总租车人数 |

weather（天气等级） -

1. 清澈，少云，多云。

2. 雾+阴天，雾+碎云、雾+少云、雾

3. 小雪、小雨+雷暴+散云，小雨+云

4. 暴雨+冰雹+雷暴+雾，雪+雾

#### 2、数据预处理：

（1）导入相关库和原始数据

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd 
from datetime import datetime
import seaborn as sns
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
sns.set(style='whitegrid' , palette='tab10')

train=pd.read_csv('../input/bike-sharing-demand/train.csv')
test=pd.read_csv('../input/bike-sharing-demand/test.csv')


（2）查看数据集的基本信息

In [ ]:
#查看训练集数据基本信息
train.info()

In [ ]:
#查看测试集数据基本信息

test.info()

（3）缺失值处理

In [ ]:
#可视化查询缺失值
import missingno as msno
msno.matrix(train,figsize=(12,5))
msno.matrix(test,figsize=(12,5))

经过观察发现，数据集中并没有缺失值，因此不需要进行缺失值处理，但数据集中忆旧有存在异常数据的可能，需要进一步分析。

（4）异常值处理

In [ ]:
#观察训练集数据描述统计
train.describe()

<1> 租赁额异常值处理

经过上一步的数据描述，我们发现租赁数量（count）的数值差异比较大，因此我么进一步处理，观察一下它们的数据密度分布。

In [ ]:
#观察租赁额密度分布
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
fig.set_size_inches(6,5)

sns.distplot(train['count'])

ax.set(xlabel='count',title='Distribution of count',)

通过观察，可以发现数据的分布比较倾斜，而且有一个比较长的尾。因此，我们通过下面的方法处理数据长尾，首先排除掉3个标准差以外的数据。

In [ ]:
train_WithoutOutliers = train[np.abs(train['count']-
                        train['count'].mean())<=(3*train['count'].std())] 
train_WithoutOutliers .shape

In [ ]:
train_WithoutOutliers['count'] .describe()

In [ ]:
#可视化比较处理前后
fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)
fig.set_size_inches(12,5)

sns.distplot(train_WithoutOutliers['count'],ax=ax1)
sns.distplot(train['count'],ax=ax2)

ax1.set(xlabel='count',title='Distribution of count without outliers',)
ax2.set(xlabel='registered',title='Distribution of count')

经过比较，我们发现数据波动依然很大，而我们希望波动相对稳定，否则容易产生过拟合，所以希望对数据进行处理，使得数据相对稳定，此处选择对数变化，来使得数据稳定。

In [ ]:
yLabels=train_WithoutOutliers['count']
yLabels_log=np.log(yLabels)
sns.distplot(yLabels_log)

通过上面的图像我们可以发现，经过对数变换后数据的分布更加均匀，大小差异也缩小了，使用这样的标签对训练模型是有效果的。

<1> 其他数值型数据异常值处理


接下来我们对其余的数值型数据进行处理，由于其他数据同时包含在两个数据集中，为方便数据处理先将两个数据集合并。

In [ ]:
Bike_data=pd.concat([train_WithoutOutliers,test],ignore_index=True)
#查看数据集大小
Bike_data.shape

In [ ]:
Bike_data.head()

通过可视化工具，查看温度、体感温度、湿度、风速这些数值型数据的密度分布。

In [ ]:
fig, axes = plt.subplots(2, 2)
fig.set_size_inches(12,10)

sns.distplot(Bike_data['temp'],ax=axes[0,0])
sns.distplot(Bike_data['atemp'],ax=axes[0,1])
sns.distplot(Bike_data['humidity'],ax=axes[1,0])
sns.distplot(Bike_data['windspeed'],ax=axes[1,1])

axes[0,0].set(xlabel='temp',title='Distribution of temp',)
axes[0,1].set(xlabel='atemp',title='Distribution of atemp')
axes[1,0].set(xlabel='humidity',title='Distribution of humidity')
axes[1,1].set(xlabel='windspeed',title='Distribution of windspeed')

通过这个分布可以发现一些问题，比如风速为0的数据比较多，而观察统计描述发现空缺值在1–6之间，从这里似乎可以推测，数据本身或许是有缺失值的，但是用0来填充了，但这些风速为0的数据会对预测产生干扰。因此我们希望使用随机森林根据相同的年份，月份，季节，温度，湿度等几个特征来填充一下风速的缺失值。填充之前看一下非零数据的描述统计。

In [ ]:
Bike_data[Bike_data['windspeed']!=0]['windspeed'].describe()

使用随机森林模型填充风速。

In [ ]:
Bike_data['date']=Bike_data.datetime.apply( lambda c : c.split( )[0])
Bike_data['hour']=Bike_data.datetime.apply( lambda c : c.split( )[1].split(':')[0]).astype('int')
Bike_data['year']=Bike_data.datetime.apply( lambda c : c.split( )[0].split('-')[0]).astype('int')
Bike_data['month']=Bike_data.datetime.apply( lambda c : c.split( )[0].split('-')[1]).astype('int')
Bike_data['weekday']=Bike_data.date.apply( lambda c : datetime.strptime(c,'%Y-%m-%d').isoweekday())
Bike_data.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

Bike_data["windspeed_rfr"]=Bike_data["windspeed"]
# 将数据分成风速等于0和不等于两部分
dataWind0 = Bike_data[Bike_data["windspeed_rfr"]==0]
dataWindNot0 = Bike_data[Bike_data["windspeed_rfr"]!=0]
#选定模型
rfModel_wind = RandomForestRegressor(n_estimators=1000,random_state=42)
# 选定特征值
windColumns = ["season","weather","humidity","month","temp","year","atemp"]
# 将风速不等于0的数据作为训练集，fit到RandomForestRegressor之中
rfModel_wind.fit(dataWindNot0[windColumns], dataWindNot0["windspeed_rfr"])
#通过训练好的模型预测风速
wind0Values = rfModel_wind.predict(X= dataWind0[windColumns])
#将预测的风速填充到风速为零的数据中
dataWind0.loc[:,"windspeed_rfr"] = wind0Values
#连接两部分数据
Bike_data = dataWindNot0.append(dataWind0)
Bike_data.reset_index(inplace=True)
Bike_data.drop('index',inplace=True,axis=1)

填充好再画图观察一下这四个特征值的密度分布。

In [ ]:
fig, axes = plt.subplots(2, 2)
fig.set_size_inches(12,10)

sns.distplot(Bike_data['temp'],ax=axes[0,0])
sns.distplot(Bike_data['atemp'],ax=axes[0,1])
sns.distplot(Bike_data['humidity'],ax=axes[1,0])
sns.distplot(Bike_data['windspeed_rfr'],ax=axes[1,1])

axes[0,0].set(xlabel='temp',title='Distribution of temp',)
axes[0,1].set(xlabel='atemp',title='Distribution of atemp')
axes[1,0].set(xlabel='humidity',title='Distribution of humidity')
axes[1,1].set(xlabel='windseed',title='Distribution of windspeed')

（5）时间型数据的处理

在这里，我们将时间型数据进一步处理，将其划分为日期、小时、年、月、日、并且进一步得到这一天是一周的星期几。

In [ ]:
Bike_data['date']=Bike_data.datetime.apply( lambda c : c.split( )[0])
Bike_data['hour']=Bike_data.datetime.apply( lambda c : c.split( )[1].split(':')[0]).astype('int')
Bike_data['year']=Bike_data.datetime.apply( lambda c : c.split( )[0].split('-')[0]).astype('int')
Bike_data['month']=Bike_data.datetime.apply( lambda c : c.split( )[0].split('-')[1]).astype('int')
Bike_data['weekday']=Bike_data.date.apply( lambda c : datetime.strptime(c,'%Y-%m-%d').isoweekday())
Bike_data.head()

#### 3、数据分析：

（1）描述性分析

In [ ]:
train.describe().T

通过观察上表，我们可以看出，温度, 体表温度, 相对湿度, 风速均近似对称分布, 而非注册用户, 注册用户,以及总数均右边分布。

In [ ]:
for i in range(5, 12):
    name = train.columns[i]
    print('{0}偏态系数为 {1}, 峰态系数为 {2}'.format(name, train[name].skew(), train[name].kurt()))

通过观察，我们发现，temp, atemp, humidity低度偏态, windspeed中度偏态, casual, registered, count高度偏态；
temp, atemp, humidity为平峰分布, windspeed,casual, registered, count为尖峰分布。


（2）探索性分析

我们要解决的问题是希望预测每一个小时的总租赁额，所以我们首先整体看一下租赁额相关的三个值与其他特征值之间的关系。

In [ ]:
sns.pairplot(Bike_data ,x_vars=['holiday','workingday','weather','season',
                                'weekday','hour','windspeed_rfr','humidity','temp','atemp'] ,
                        y_vars=['casual','registered','count'] , plot_kws={'alpha': 0.1})

经过观察，可以得到以下结论：-

1. 已经注册的用户在工作日出行较多，在节假日出行较少，而临时（非注册）用户则相反；
2. 在第一季度，使用共享单车出行的人数总体偏少；
3. 租赁数量随天气等级上升而减少，即天气越恶劣，共享单车租赁量越少；
4. 小时数对租赁情况影响明显，注册用户呈现两个高峰（大概是每天的早高峰和晚高峰），非注册用户呈现一个正态分布；
5. 租赁数量随风速增大而减少；
6. 温度、湿度对非注册用户影响比较大，对注册用户影响较小

（3）相关性分析

查看各个特征与每小时租车总量（count）的相关性，由于上图可以看出特征值与租车数量基本是线性相关，所以求他们的线性相关系数。

In [ ]:
#相关性矩阵
corrDf = Bike_data.corr() 

#ascending=False表示按降序排列
corrDf['count'].sort_values(ascending =False)

将结果进行可视化处理。

In [ ]:
correlation = Bike_data.corr()
mask = np.array(correlation)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(correlation, mask=mask,vmax=.8, square=True,annot=True)

plt.show()

通过上图，我们可以得到以下结论: 


count 和 registered、casual高度正相关，相关系数分别为0.7 与0.97。因为 count = casual + registered ，所以这个正相关和预期相符。count 和 temp 正相关，相关系数为 0.39。一般来说，气温过低人们不愿意骑车出行。count 和 humidity（湿度）负相关，湿度过大的天气不适宜骑车。当然考虑湿度的同时也应该考虑温度。windspeed似乎对租车人数影响不大（0.1），但我们也应该考虑到极端大风天气出现频率应该不高。风速在正常范围内波动应该对人们租车影响不大。可以看出特征值对租赁数量的影响力度为,时段>温度>湿度>年份>月份>季节>天气等级>风速>星期几>是否工作日>是否假日。

（4）影响因素分析

<1> 分析时段对租赁数量的影响

In [ ]:
workingday_df=Bike_data[Bike_data['workingday']==1]
workingday_df = workingday_df.groupby(['hour'], as_index=True).agg({'casual':'mean',
                                                                    'registered':'mean',
                                                                    'count':'mean'})

nworkingday_df=Bike_data[Bike_data['workingday']==0]
nworkingday_df = nworkingday_df.groupby(['hour'], as_index=True).agg({'casual':'mean',
                                                                      'registered':'mean', 
                                                                      'count':'mean'})
fig, axes = plt.subplots(1, 2,sharey = True)

workingday_df.plot(figsize=(15,5),title = 'The average number of rentals initiated per hour in the working day',ax=axes[0])
nworkingday_df.plot(figsize=(15,5),title = 'The average number of rentals initiated per hour in the nonworkdays',ax=axes[1])

通过对比可以看出：

1. 工作日对于非注册用户上下班时间是两个用车高峰，而中午也会有一个小高峰，猜测可能是外出午餐的人；
2. 而对非注册用户起伏比较平缓，高峰期在17点左右；
3. 并且注册用户的用车数量远超过非注册用户。
4. 对非工作日而言租赁数量随时间呈现一个正态分布，高峰在14点左右，低谷在4点左右，且分布比较均匀。

<2>温度对租赁数量的影响

观察温度的走势

In [ ]:
#按天汇总取一天的气温中位数
temp_df = Bike_data.groupby(['date','weekday'], as_index=False).agg({'year':'mean',
                                                                     'month':'mean',
                                                                     'temp':'median'})
#由于测试数据集中没有租赁信息，会导致折线图有断裂，所以将缺失的数据丢弃
temp_df.dropna ( axis = 0 , how ='any', inplace = True )

#预计按天统计的波动仍然很大，再按月取日平均值
temp_month = temp_df.groupby(['year','month'], as_index=False).agg({'weekday':'min',
                                                                    'temp':'median'})

#将按天求和统计数据的日期转换成datetime格式
temp_df['date']=pd.to_datetime(temp_df['date'])

#将按月统计数据设置一列时间序列
temp_month.rename(columns={'weekday':'day'},inplace=True)
temp_month['date']=pd.to_datetime(temp_month[['year','month','day']])

#设置画框尺寸
fig = plt.figure(figsize=(18,6))
ax = fig.add_subplot(1,1,1)

#使用折线图展示总体租赁情况（count）随时间的走势
plt.plot(temp_df['date'] , temp_df['temp'] , linewidth=1.3 , label='Daily average')
ax.set_title('Change trend of average temperature per day in two years')
plt.plot(temp_month['date'] , temp_month['temp'] , marker='o', linewidth=1.3 ,
         label='Monthly average')
ax.legend()


可以看出每年的气温趋势相同随月份变化，在7月份气温最高，1月份气温最低，再看一下每小时平均租赁数量随温度变化的趋势。

In [ ]:
#按温度取租赁额平均值
temp_rentals = Bike_data.groupby(['temp'], as_index=True).agg({'casual':'mean', 
                                                               'registered':'mean',
                                                               'count':'mean'})
temp_rentals .plot(title = 'The average number of rentals initiated per hour changes with the temperature')

可观察到随气温上升租车数量总体呈现上升趋势，但在气温超过35时开始下降，在气温4度时达到最低点。



<3>湿度对租赁数量的影响

观察湿度的走势：

In [ ]:
humidity_df = Bike_data.groupby('date', as_index=False).agg({'humidity':'mean'})
humidity_df['date']=pd.to_datetime(humidity_df['date'])
#将日期设置为时间索引
humidity_df=humidity_df.set_index('date')

humidity_month = Bike_data.groupby(['year','month'], as_index=False).agg({'weekday':'min',
                                                                          'humidity':'mean'})
humidity_month.rename(columns={'weekday':'day'},inplace=True)
humidity_month['date']=pd.to_datetime(humidity_month[['year','month','day']])

fig = plt.figure(figsize=(18,6))
ax = fig.add_subplot(1,1,1)
plt.plot(humidity_df.index , humidity_df['humidity'] , linewidth=1.3,label='Daily average')
plt.plot(humidity_month['date'], humidity_month['humidity'] ,marker='o', 
         linewidth=1.3,label='Monthly average')
ax.legend()
ax.set_title('Change trend of average humidity per day in two years')

通过上图，我们发现湿度的变化幅度不是很大，多数围绕60上下浮动，本次数据范围内峰值为80。

In [ ]:
humidity_rentals = Bike_data.groupby(['humidity'], as_index=True).agg({'casual':'mean',
                                                                       'registered':'mean',
                                                                       'count':'mean'})
humidity_rentals .plot (title = 'Average number of rentals initiated per hour in different humidity')

可以观察到在湿度20左右租赁数量迅速达到高峰值，此后缓慢递减。

<4>年份、月份对租赁数量的影响

观察两年时间里，总租车数量随时间变化的趋势.

In [ ]:
#数据按小时统计展示起来太麻烦，希望能够按天汇总
count_df = Bike_data.groupby(['date','weekday'], as_index=False).agg({'year':'mean',
                                                                      'month':'mean',
                                                                      'casual':'sum',
                                                                      'registered':'sum',
                                                                       'count':'sum'})
#由于测试数据集中没有租赁信息，会导致折线图有断裂，所以将缺失的数据丢弃
count_df.dropna ( axis = 0 , how ='any', inplace = True )

#预计按天统计的波动仍然很大，再按月取日平均值
count_month = count_df.groupby(['year','month'], as_index=False).agg({'weekday':'min',
                                                                      'casual':'mean', 
                                                                      'registered':'mean',
                                                                      'count':'mean'})

#将按天求和统计数据的日期转换成datetime格式
count_df['date']=pd.to_datetime(count_df['date'])

#将按月统计数据设置一列时间序列
count_month.rename(columns={'weekday':'day'},inplace=True)
count_month['date']=pd.to_datetime(count_month[['year','month','day']])

#设置画框尺寸
fig = plt.figure(figsize=(18,6))
ax = fig.add_subplot(1,1,1)

#使用折线图展示总体租赁情况（count）随时间的走势
plt.plot(count_df['date'] , count_df['count'] , linewidth=1.3 , label='Daily average')
ax.set_title('Change trend of average number of rentals initiated  per day in two years')
plt.plot(count_month['date'] , count_month['count'] , marker='o', 
         linewidth=1.3 , label='Monthly average')
ax.legend()

通过上图，可以看出：

1. 共享单车的租赁情况2012年整体是比2011年有增涨的；
2. 租赁情况随月份波动明显；
3. 数据在2011年9到12月，2012年3到9月间波动剧烈；
4. 有很多局部波谷值。

<5>季节对出行人数的影响

In [ ]:
day_df=Bike_data.groupby('date').agg({'year':'mean','season':'mean',
                                      'casual':'sum', 'registered':'sum'
                                      ,'count':'sum','temp':'mean',
                                      'atemp':'mean'})
season_df = day_df.groupby(['year','season'], as_index=True).agg({'casual':'mean', 
                                                                  'registered':'mean',
                                                                  'count':'mean'})
temp_df = day_df.groupby(['year','season'], as_index=True).agg({'temp':'mean', 
                                                                'atemp':'mean'})
season_df .plot(figsize=(18,6),title = 'The trend of average number of rentals initiated per day changes with season')

In [ ]:
temp_df = day_df.groupby(['year','season'], as_index=True).agg({'temp':'mean', 
                                                                'atemp':'mean'})
temp_df.plot(figsize=(18,6),title = 'The trend of average temperature per day changes with season')

可以看出无论是非注册用户还是注册用户用车的数量都在秋季迎来高峰，而春季度用户数量最低。

<6>天气情况对出行情况的影响

观察天气情况的统计情况

In [ ]:
count_weather = Bike_data.groupby('weather')
count_weather[['casual','registered','count']].count()

考虑到不同天气的天数不同，例如非常糟糕的天气（4）会很少出现，查看一下不同天气等级的数据条数，再对租赁数量按天气等级取每小时平均值。

In [ ]:
weather_df = Bike_data.groupby('weather', as_index=True).agg({'casual':'mean',
                                                              'registered':'mean'})
weather_df.plot.bar(stacked=True,title = 'Average number of rentals initiated per hour in different weather')

在这里，我们发现了不合理的数据：即天气等级4的时候出行人数并不少，尤其是会员出行人数甚至比天气等级2的平均值还高，按理说4等级的应该是最少的，将天气等级4的数据打印出来找一下原因：

In [ ]:
Bike_data[Bike_data['weather']==4]

观察可知该数据是在上下班高峰期产生的，所以该数据是个异常数据。不具有代表性。

<7>风速对出行情况的影响

In [ ]:
windspeed_df = Bike_data.groupby('date', as_index=False).agg({'windspeed_rfr':'mean'})
windspeed_df['date']=pd.to_datetime(windspeed_df['date'])
#将日期设置为时间索引
windspeed_df=windspeed_df.set_index('date')

windspeed_month = Bike_data.groupby(['year','month'], as_index=False).agg({'weekday':'min',
                                                                           'windspeed_rfr':'mean'})
windspeed_month.rename(columns={'weekday':'day'},inplace=True)
windspeed_month['date']=pd.to_datetime(windspeed_month[['year','month','day']])

fig = plt.figure(figsize=(18,6))
ax = fig.add_subplot(1,1,1)
plt.plot(windspeed_df.index , windspeed_df['windspeed_rfr'] , linewidth=1.3,label='Daily average')
plt.plot(windspeed_month['date'], windspeed_month['windspeed_rfr'] ,
         marker='o', linewidth=1.3,label='Monthly average')
ax.legend()
ax.set_title('Change trend of average number of windspeed  per day in two years')

可以看出风速在2011年9月份和2011年12月到2012年3月份间波动和大，观察一下租赁人数随风速变化趋势，考虑到风速特别大的时候很少，如果取平均值会出现异常，所以按风速对租赁数量取最大值。



In [ ]:
windspeed_rentals = Bike_data.groupby(['windspeed'], as_index=True).agg({'casual':'max', 
                                                                         'registered':'max',
                                                                         'count':'max'})
windspeed_rentals .plot(title = 'Max number of rentals initiated per hour in different windspeed')

通过观察，可以发现租赁数量随风速越大租赁数量越少，在风速超过30的时候明显减少，但风速在风速40左右却有一次反弹，打印数据找一下反弹原因：

In [ ]:
df2=Bike_data[Bike_data['windspeed']>40]
df2=df2[df2['count']>400]
df2

该条数据产生在上下班高峰期时期，所以也是个异常值，不具有代表性。

<8>日期对出行情况的影响

考虑到相同日期是否工作日，星期几，以及所属年份等信息是一样的，把租赁数据按天求和，其它日期类数据取平均值

In [ ]:
day_df = Bike_data.groupby(['date'], as_index=False).agg({'casual':'sum','registered':'sum',
                                                          'count':'sum', 'workingday':'mean',
                                                          'weekday':'mean','holiday':'mean',
                                                          'year':'mean'})
day_df.head()

In [ ]:
number_pei=day_df[['casual','registered']].mean()
number_pei

In [ ]:
plt.axes(aspect='equal')  
plt.pie(number_pei, labels=['casual','registered'], autopct='%1.1f%%', 
        pctdistance=0.6 , labeldistance=1.05 , radius=1 )  
plt.title('Casual or registered in the total lease')

通过饼图可以发现，大部分的用户都是注册用户，只有不到二成用户为非注册用户。

下面我们分为工作日和休息日来观察日期对出行情况的影响。

1）工作日

由于工作日和休息日的天数差别，对工作日和非工作日租赁数量取了平均值，对一周中每天的租赁数量求和

In [ ]:
workingday_df=day_df.groupby(['workingday'], as_index=True).agg({'casual':'mean', 
                                                                 'registered':'mean'})
workingday_df_0 = workingday_df.loc[0]
workingday_df_1 = workingday_df.loc[1]

# plt.axes(aspect='equal')
fig = plt.figure(figsize=(8,6)) 
plt.subplots_adjust(hspace=0.5, wspace=0.2)     #设置子图表间隔
grid = plt.GridSpec(2, 2, wspace=0.5, hspace=0.5)   #设置子图表坐标轴 对齐

plt.subplot2grid((2,2),(1,0), rowspan=2)
width = 0.3       # 设置条宽

p1 = plt.bar(workingday_df.index,workingday_df['casual'], width)
p2 = plt.bar(workingday_df.index,workingday_df['registered'], 
             width,bottom=workingday_df['casual'])
plt.title('Average number of rentals initiated per day')
plt.xticks([0,1], ('nonworking day', 'working day'),rotation=20)
plt.legend((p1[0], p2[0]), ('casual', 'registered'))

plt.subplot2grid((2,2),(0,0))
plt.pie(workingday_df_0, labels=['casual','registered'], autopct='%1.1f%%', 
        pctdistance=0.6 , labeldistance=1.35 , radius=1.3)
plt.axis('equal') 
plt.title('nonworking day')

plt.subplot2grid((2,2),(0,1))
plt.pie(workingday_df_1, labels=['casual','registered'], autopct='%1.1f%%', 
        pctdistance=0.6 , labeldistance=1.35 , radius=1.3)
plt.title('working day')
plt.axis('equal') 

In [ ]:
weekday_df= day_df.groupby(['weekday'], as_index=True).agg({'casual':'mean', 'registered':'mean'})
weekday_df.plot.bar(stacked=True , title = 'Average number of rentals initiated per day by weekday')

通过对比图，我们可以发现：

1. 工作日会员用户出行数量较多，临时用户出行数量较少；
2. 周末会员用户租赁数量降低，临时用户租赁数量增加。

2）节假日

由于节假日在一年中数量占比非常少，先来看一每年的节假日下有几天：

In [ ]:
holiday_coun=day_df.groupby('year', as_index=True).agg({'holiday':'sum'})
holiday_coun

假期的天数占一年天数的份额十分少，所以对假期和非假期取日平均值。

In [ ]:
holiday_df = day_df.groupby('holiday', as_index=True).agg({'casual':'mean', 'registered':'mean'})
holiday_df.plot.bar(stacked=True , title = 'Average number of rentals initiated per day by holiday or not')

通过分析，我们发现，节假日会员或非会员使用量都比非节假日多，符合客观规律。

### 三、特征工程

根据前面的观察，我们决定将时段（hour）、温度（temp）、湿度（humidity）、年份（year）、月份（month）、季节（season）、天气等级（weather）、风速（windspeed_rfr）、星期几（weekday）、是否工作日（workingday）、是否假日（holiday），这11项作为特征值。由于CART决策树使用二分类，所以将多类别型数据使用one-hot转化成多个二分型类别

In [ ]:
dummies_month = pd.get_dummies(Bike_data['month'], prefix= 'month')
dummies_season=pd.get_dummies(Bike_data['season'],prefix='season')
dummies_weather=pd.get_dummies(Bike_data['weather'],prefix='weather')
dummies_year=pd.get_dummies(Bike_data['year'],prefix='year')
#把5个新的DF和原来的表连接起来
Bike_data=pd.concat([Bike_data,dummies_month,dummies_season,dummies_weather,dummies_year],axis=1)

### 四、模型构建

#### 1、分离训练集和测试集

In [ ]:
dataTrain = Bike_data[pd.notnull(Bike_data['count'])]
dataTest= Bike_data[~pd.notnull(Bike_data['count'])].sort_values(by=['datetime'])
datetimecol = dataTest['datetime']
yLabels=dataTrain['count']
yLabels_log=np.log(yLabels)

#### 2、舍弃多余的特征值

In [ ]:
dropFeatures = ['casual' , 'count' , 'datetime' , 'date' , 'registered' ,
                'windspeed' , 'atemp' , 'month','season','weather', 'year' ]

dataTrain = dataTrain.drop(dropFeatures , axis=1)
dataTest = dataTest.drop(dropFeatures , axis=1)


#### 3、训练模型

这里我们使用的是随机森林模型，随机森林是一个包含多个决策树的分类器， 并且其输出的类别是由个别树输出的类别的众数而定。

In [ ]:
rfModel = RandomForestRegressor(n_estimators=1000 , random_state = 42)

rfModel.fit(dataTrain , yLabels_log)

preds = rfModel.predict( X = dataTrain)

#### 4、预测测试集数据

In [ ]:
predsTest= rfModel.predict(X = dataTest)

submission=pd.DataFrame({'datetime':datetimecol , 'count':[max(0,x) for x in np.exp(predsTest)]})

submission.to_csv('./bike_predictions.csv',index=False)